# Первоначальная частица. 
Поиск зависимостей для выявления первоначальной частицы.  
Что если попробовать просто на модельных данных предсказать тип частицы, которая участвовала во взаимодействии.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from scipy import stats
import datetime
import math

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [ ]:
# # Экспериментальные данные.
# AllExp = pd.read_csv('datachanged/AllExpC')
# AllExp = AllExp.drop(['Unnamed: 0'], axis = 1)
# AllExp.columns
# AllExpNew = pandas.read_csv('datachanged/AllExpNewL')

In [242]:
# Новый модельный банк.
AllMc0 = pd.read_csv('datachanged/AllMc0CNewBank', sep = '\t')
# Семейчтва в новом банке были перенумерованы, поэтому объединять нужно по-другому
newE = AllMc0[['num', ' E0', ' A0', 'cnt']].drop_duplicates()
AllMc0New = pd.read_csv('datachanged/AllMc0NewLNewBank').\
    drop(['Unnamed: 0'], axis = 1)[['num', 'n_gamma', 'n_gamma_thr', 'sum_energy_thr']]
AllMc0 = AllMc0.merge(AllMc0New, left_on='num', right_on='num')
print(AllMc0.columns)
AllMc0[' A0'].value_counts()

Index([' j', 'X(J)', 'Y(J)', 'E(J)', 'H(J)', ' E0', ' A0', 'log_E0', 'num',
       'R', 'ER', 'sum_energy', 'lg_r', 'lg_e_r', 'cnt', 'n_gamma',
       'n_gamma_thr', 'sum_energy_thr'],
      dtype='object')


1    69331
2    18964
9    10197
6     5088
4     2685
5     2466
7     2046
8     1620
3      370
Name:  A0, dtype: int64

# Старый модельный банк.

In [266]:
# # Старый модельный банк.
AllMc0 = pd.read_csv('datachanged/AllMc0COld', sep = '\t')
AllMc0.columns

Index([' num_of_family', ' j', 'X(J)', 'Y(J)', 'E(J)', 'H(J)', ' E0', ' A0',
       'log_E0', 'R', 'ER', 'sum_energy'],
      dtype='object')

In [116]:
AllMc0['lg_r'] = np.log(AllMc0['R'])
AllMc0['lg_e_r'] = np.log(AllMc0['ER'])
AllMc0['cnt'] = AllMc0.groupby([' num_of_family'])[' num_of_family'].transform('count')

In [117]:
AllMc0.columns

Index([' num_of_family', ' j', 'X(J)', 'Y(J)', 'E(J)', 'H(J)', ' E0', ' A0',
       'log_E0', 'R', 'ER', 'sum_energy', 'lg_r', 'lg_e_r', 'cnt'],
      dtype='object')

In [265]:
AllMc0New = pd.read_csv('datachanged/AllMc0NewL').\
    drop(['Unnamed: 0'], axis = 1)[[' num_of_family', 'n_gamma', 'n_gamma_thr', 'sum_energy_thr']]
AllMc0New.columns

Index([' num_of_family', 'n_gamma', 'n_gamma_thr', 'sum_energy_thr'], dtype='object')

In [190]:
AllMc0 = AllMc0.merge(AllMc0New, left_on=' num_of_family', right_on=' num_of_family')
AllMc0.columns

Index([' num_of_family', ' j', 'X(J)', 'Y(J)', 'E(J)', 'H(J)', ' E0', ' A0',
       'log_E0', 'R', 'ER', 'sum_energy', 'lg_r', 'lg_e_r', 'cnt', 'n_gamma',
       'n_gamma_thr', 'sum_energy_thr'],
      dtype='object')

In [191]:
AllMc0.head()

,num_of_family,j,X(J),Y(J),E(J),H(J),E0,A0,log_E0,R,ER,sum_energy,lg_r,lg_e_r,cnt,n_gamma,n_gamma_thr,sum_energy_thr
0,1,1,-3.013189,-7.227429,4.611031,1255.250,3366.712,1,3.527206,7.830392,36.106180,370.199507,2.058013,3.586464,26,26,4,239
1,1,2,-1.310529,-2.563248,12.047700,1545.054,3366.712,1,3.527206,2.878841,34.683415,370.199507,1.057388,3.546262,26,26,4,239
2,1,3,-2.178048,-2.144561,5.401844,3044.520,3366.712,1,3.527206,3.056638,16.511481,370.199507,1.117316,2.804056,26,26,4,239
3,1,4,-0.965465,-0.082880,5.325506,4715.523,3366.712,1,3.527206,0.969016,5.160501,370.199507,-0.031474,1.641034,26,26,4,239
4,1,5,-1.621645,0.079540,15.935060,3044.520,3366.712,1,3.527206,1.623595,25.872076,370.199507,0.484643,3.253164,26,26,4,239


In [60]:
# proton = AllMc0[AllMc0[' A0'] == 1].copy() # протоны
# ferrum = AllMc0[AllMc0[' A0'] == 9].copy() # железо
# CNO = AllMc0[(AllMc0[' A0']>=6)&(AllMc0[' A0'] <= 8)].copy() # CNO которое на самом деле MgSi

## Можно ли считать A0 для конкретной частицы.

In [243]:
# Нужно разделить так, чтобы частицы из одного семейства не заглядывали в частицы из другого семейства
num_name = ' num_of_family'
num_name = 'num'

particles = AllMc0[' A0'].unique()

train = []
test = []

for particle in particles:
    # Выделяем тип частицы
    df_tmp = AllMc0[AllMc0[' A0']==particle].copy()
    df_tmp = df_tmp.sort_values(by = [num_name, ' j'])
    sample = int(len(df_tmp)*0.6)
    split = max(df_tmp[:sample][num_name])
    df_train = df_tmp[df_tmp[num_name]<split].copy()
    df_test = df_tmp[df_tmp[num_name]>=split].copy()
    print(f'shape {particle} train: {len(df_train)} test: {len(df_test)}')
    train.append(df_train)
    test.append(df_test)
train = pd.concat(train)
test = pd.concat(test)

shape 1 train: 41496 test: 27835
shape 2 train: 11048 test: 7916
shape 4 train: 1582 test: 1103
shape 3 train: 188 test: 182
shape 9 train: 6053 test: 4144
shape 5 train: 1284 test: 1182
shape 6 train: 2596 test: 2492
shape 7 train: 1212 test: 834
shape 8 train: 435 test: 1185


In [245]:
# Нужно разделить так, чтобы частицы из одного семейства не заглядывали в частицы из другого семейства
num_name = ' num_of_family'
num_name = 'num'
particles = AllMc0[' A0'].unique()

train = []
test = []

for particle in particles:
    # Выделяем тип частицы
    df_tmp = AllMc0[AllMc0[' A0']==particle].copy()
    df_tmp = df_tmp.sort_values(by = [num_name, ' j'])
    sample = int(len(df_tmp)*0.6)
    split = max(df_tmp[:sample][num_name])
    df_train = df_tmp[df_tmp[num_name]<split].copy()
    df_test = df_tmp[df_tmp[num_name]>=split].copy()
    
    split2 = max(df_train[:1000][num_name])
    df_train = df_train[df_train[num_name]<=split2].copy()
    
    split2 = max(df_test[:1000][num_name])
    df_test = df_test[df_test[num_name]<=split2].copy()
    
    print(f'shape {particle} train: {len(df_train)} test: {len(df_test)}')
    train.append(df_train)
    test.append(df_test)
train = pd.concat(train)
test = pd.concat(test)

shape 1 train: 1006 test: 1005
shape 2 train: 1048 test: 1010
shape 4 train: 1000 test: 1009
shape 3 train: 188 test: 182
shape 9 train: 1064 test: 1008
shape 5 train: 1016 test: 1025
shape 6 train: 1211 test: 2063
shape 7 train: 1018 test: 834
shape 8 train: 435 test: 1185


In [247]:
X_train = train.drop([num_name, ' j', ' A0', ' E0', 'log_E0', 'H(J)'], axis = 1).copy()
X_test = test.drop([num_name, ' j', ' A0', ' E0', 'log_E0', 'H(J)'], axis = 1).copy()
y_train = train[' A0'].values
y_test = test[' A0'].values
num_train = train[num_name]
num_test = test[num_name]

In [248]:
%%time
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test) 

print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

           1       0.26      0.09      0.13      1005
           2       0.00      0.00      0.00      1010
           3       0.00      0.00      0.00       182
           4       0.15      0.18      0.17      1009
           5       0.02      0.03      0.02      1025
           6       0.48      1.00      0.65      2063
           7       0.11      0.04      0.06       834
           8       0.00      0.00      0.00      1185
           9       0.17      0.16      0.17      1008

    accuracy                           0.27      9321
   macro avg       0.13      0.17      0.13      9321
weighted avg       0.18      0.27      0.20      9321

Wall time: 27.3 s


In [249]:
df_test = test[[num_name, ' A0']].copy()
df_test['y_pred'] = y_pred

In [250]:
df_test[df_test[' A0']!=1][num_name].unique()

array([1588, 1598, 1603, 1592, 1643, 1689, 1707, 1760, 1865, 1879, 1902,
       1970, 1983, 2000, 2015, 2054, 2084, 2156, 2194, 2196, 2198, 2204,
       2237, 2256, 2266, 2274, 1347, 1577, 1719, 1725, 1852, 2219, 1627,
       1649, 1651, 1659, 1661, 1665, 1666, 1674, 1681, 1684, 1690, 1715,
       1727, 1729, 1754, 1664, 1722, 1749, 1759, 1779, 1786, 1789, 1830,
       1922, 1994, 2023, 2079, 2140, 2144, 2146, 2174, 2230, 2246, 1755,
       1981, 1996, 2036, 2042, 2053, 2085, 2087, 2091, 2155, 2166, 2182,
       2265, 2281, 2202], dtype=int64)

In [252]:
XX = test.groupby(num_name).mean().reset_index()
yy = XX[' A0'].values
num_yy = XX[num_name].values
XX = XX.drop([' j', num_name,' A0', ' E0', 'log_E0', 'H(J)'], axis = 1).copy()

In [253]:
yy_pred = clf.predict(XX) 
print(classification_report(yy, yy_pred, zero_division = 0))

              precision    recall  f1-score   support

         1.0       0.21      0.16      0.18        19
         2.0       0.00      0.00      0.00         3
         3.0       0.00      0.00      0.00         6
         4.0       0.27      0.17      0.21        23
         5.0       0.05      0.06      0.05        18
         6.0       0.17      1.00      0.29         1
         7.0       0.00      0.00      0.00        13
         8.0       0.00      0.00      0.00         1
         9.0       0.11      0.13      0.12        15

    accuracy                           0.11        99
   macro avg       0.09      0.17      0.09        99
weighted avg       0.13      0.11      0.11        99



In [258]:
df = XX.copy()
df[num_name] = num_yy
df[' A0'] = np.int32(yy)
df['y_pred'] = yy_pred

In [256]:
df[df[num_name]==705][[' A0', 'y_pred']]

In [259]:
df[df[num_name]==2166][[' A0', 'y_pred']]

,A0,y_pred
82,7,6


In [260]:
df[df[num_name]==873][[' A0', 'y_pred']]

,A0,y_pred


In [262]:
df[df[num_name]==1689][[' A0', 'y_pred']]

,A0,y_pred
37,4,5


In [261]:
df[df[num_name]==885][[' A0', 'y_pred']]

,A0,y_pred


In [263]:
df[df[num_name]==2084][[' A0', 'y_pred']]

,A0,y_pred
73,4,1


## A0 для семейства

In [269]:
# Новый модельный банк.
AllMc0 = pd.read_csv('datachanged/AllMc0CNewBank', sep = '\t')
# Семейчтва в новом банке были перенумерованы, поэтому объединять нужно по-другому
newE = AllMc0[['num', ' E0', ' A0', 'cnt']].drop_duplicates()
AllMc0New = pd.read_csv('datachanged/AllMc0NewLNewBank').\
    drop(['Unnamed: 0'], axis = 1)
AllMc0New = AllMc0New.merge(newE, left_on='num', right_on='num')
AllMc0New.columns

Index(['num', 'sum_energy', 'mean_r', 'mean_e_r', 'lg_mean_r', 'lg_mean_er',
       'n_gamma', 'n_gamma_thr', 'sum_energy_thr', ' E0', ' A0', 'cnt'],
      dtype='object')

In [62]:
# Первичная энергия событий в старой базе
oldE = pd.read_csv('newdata/E0_BCR.txt')
oldE = oldE.loc[oldE[' num_of_family']<1299].copy() # убираем дубли
AllMc0New = pd.read_csv('datachanged/AllMc0NewL').drop(['Unnamed: 0'], axis = 1)
AllMc0New = AllMc0New.merge(oldE, left_on=' num_of_family', right_on=' num_of_family')
AllMc0New.columns

Index([' num_of_family', 'sum_energy', 'mean_r', 'mean_e_r', 'lg_mean_r',
       'lg_mean_er', 'n_gamma', 'n_gamma_thr', 'sum_energy_thr', ' E0', ' A0'],
      dtype='object')

In [284]:
# Разделение такое, чтобы семейств с 1 было не слишком много
num_name = ' num_of_family'
num_name = 'num'
particles = AllMc0New[' A0'].unique()

train = []
test = []

for particle in particles:
    # Выделяем тип частицы
    df_tmp = AllMc0New[AllMc0New[' A0']==particle].copy()
    df_tmp = df_tmp.sort_values(by = [num_name])
    sample = int(len(df_tmp)*0.6)
    split = max(df_tmp[:sample][num_name])
    df_train = df_tmp[df_tmp[num_name]<split].copy()
    df_test = df_tmp[df_tmp[num_name]>=split].copy()
    
    split2 = max(df_train[:300][num_name])
    df_train = df_train[df_train[num_name]<=split2].copy()
    
    split2 = max(df_test[:300][num_name])
    df_test = df_test[df_test[num_name]<=split2].copy()
    
    print(f'shape {particle} train: {len(df_train)} test: {len(df_test)}')
    train.append(df_train)
    test.append(df_test)
train = pd.concat(train)
test = pd.concat(test)

shape 1 train: 300 test: 300
shape 2 train: 210 test: 143
shape 4 train: 39 test: 29
shape 3 train: 6 test: 7
shape 9 train: 95 test: 65
shape 5 train: 33 test: 25
shape 6 train: 30 test: 23
shape 7 train: 30 test: 23
shape 8 train: 7 test: 7


In [280]:
X_train = train.drop([num_name, ' A0', ' E0'], axis = 1).copy()
X_test = test.drop([num_name, ' A0', ' E0'], axis = 1).copy()
y_train = train[' A0'].values
y_test = test[' A0'].values
num_train = train[num_name]
num_test = test[num_name]

In [281]:
clf2 = GradientBoostingClassifier(random_state=0)
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test) 

print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

           1       0.37      0.12      0.18       200
           2       0.31      0.68      0.42       143
           3       0.00      0.00      0.00         7
           4       0.00      0.00      0.00        29
           5       0.00      0.00      0.00        25
           6       0.08      0.04      0.06        23
           7       0.09      0.04      0.06        23
           8       0.00      0.00      0.00         7
           9       0.36      0.49      0.42        65

    accuracy                           0.30       522
   macro avg       0.13      0.15      0.13       522
weighted avg       0.28      0.30      0.24       522



Результаты показывают, что данных по частицам недостаточно для того чтобы предположить к какому типу относится частица.  